In [ ]:
! gdown --id 1BGbi1w1UyWX7q9kfiJg4GC_wvBql1DJi /content/drive/MyDrive/CL_Cup_IT_Data_Scince_секция_кейс_VK_датасет.zip

usage: gdown
       [-h]
       [-V]
       [-O OUTPUT]
       [-q]
       [--fuzzy]
       [--id]
       [--proxy PROXY]
       [--speed SPEED]
       [--no-cookies]
       [--no-check-certificate]
       [--continue]
       [--folder]
       [--remaining-ok]
       url_or_id
gdown: error: unrecognized arguments: /content/drive/MyDrive/CL_Cup_IT_Data_Scince_секция_кейс_VK_датасет.zip


In [ ]:
!unzip /content/drive/MyDrive/CL_Cup_IT_Data_Scince_секция_кейс_VK_датасет.zip

Archive:  /content/drive/MyDrive/CL_Cup_IT_Data_Scince_секция_кейс_VK_датасет.zip
  inflating: readme.txt              
  inflating: ranking_train.jsonl     
  inflating: ranking_test.jsonl      


# Обработка test

In [53]:
import json
import re 
from html import unescape
import pandas as pd

data = []
# Создаем словарь, в котором ключи - это регулярные выражения для поиска смайликов,
# а значения - это строки замены, которые будут использоваться для замены найденных смайликов
smileys = {
    r"[;:]-?[)D\]]": " SMILE ",  # положительные смайлики (;-), ;-D, :-), :-D, :), : D)
    r"[;:]-?\(" : " SAD "  # грустные смайлики
}
with open('/content/ranking_test.jsonl', 'r', encoding='ascii') as f:
    for line in f:
        json_data = json.loads(line.strip())
        post_text = unescape(json_data['text'])
        post_text = post_text.replace('\'', '').replace('\n', '').replace('\t', '')
        post_text = re.sub(r'[^\x00-\x7F]+', '', post_text)
        comments = json_data['comments']
        for comment in comments:
            comment_text = unescape(comment['text'])
            comment_text = re.sub("'", '"', comment_text) #заменяю ' на ", чтобы убрать экранирование \'
            comment_text = comment_text.replace('"', "'") #обратная замена
            for pattern, replacement in smileys.items():
                comment_text = re.sub(pattern, replacement, comment_text)
            # Используем регулярное выражение для поиска всех ссылок в тексте
            urls = re.findall(r'(http\S+?)(?=(http|\s|$))', comment_text) 
            # Используем цикл, чтобы заменить каждую ссылку в тексте на эту же ссылку с пробелами по обе стороны
            for url in urls:
                comment_text = comment_text.replace(url[0], f' {url[0]} ')
            comment_text = re.sub(r'[^\x00-\x7F]+', '', comment_text) #убирает непонятные символы unicode 
            comment_text = re.sub(r'\s+', ' ',  comment_text.strip()) #убирает все лишние пробелы
            comment_text = re.sub(r'(https?://\S+|www\.\S+)', 'URL', comment_text) #заменяет ссылки http,https, www на URL
            #comment_text = re.sub(r'[^\w\s]|_', '', comment_text) # очистка текста от всех символов, которые не являются алфавитно-цифровыми, лишних пробелов и знаков препинания
            score = comment.get('score')
            data.append({'post_text': post_text, 'comment': comment_text , 'score': score})

df_test = pd.DataFrame(data)

In [54]:
df_test

,post_text,comment,score
0,"iOS 8.0.1 released, broken on iPhone 6 models,...",I'm still waiting for them to stabilize wifi o...,None
1,"iOS 8.0.1 released, broken on iPhone 6 models,...","For those who upgraded, no need to do a restor...",None
2,"iOS 8.0.1 released, broken on iPhone 6 models,...",Upgraded shortly after it was released and suf...,None
3,"iOS 8.0.1 released, broken on iPhone 6 models,...",I think they were under a lot of pressure on t...,None
4,"iOS 8.0.1 released, broken on iPhone 6 models,...",Fix for those who already updated: URL,None
...,...,...,...
70015,Why does Gmail hate my domain?,I send a LOT of emails each month (email newsl...,None
70016,Why does Gmail hate my domain?,I hit a similar problems when sending automate...,None
70017,Why does Gmail hate my domain?,That's all a bit presumptive and inflammatory ...,None
70018,Why does Gmail hate my domain?,If the domain is bitbin.de and the mail server...,None


# Обработка train

In [55]:
import json
import re 
from html import unescape
import pandas as pd

data = []
# Создаем словарь, в котором ключи - это регулярные выражения для поиска смайликов,
# а значения - это строки замены, которые будут использоваться для замены найденных смайликов
smileys = {
    r"[;:]-?[)D\]]": " SMILE ",  # положительные смайлики (;-), ;-D, :-), :-D, :), : D)
    r"[;:]-?\(" : " SAD "  # грустные смайлики
}
with open('/content/ranking_train.jsonl', 'r', encoding='ascii') as f:
    for line in f:
        json_data = json.loads(line.strip())
        post_text = unescape(json_data['text'])
        post_text = post_text.replace('\'', '').replace('\n', '').replace('\t', '')
        post_text = re.sub(r'[^\x00-\x7F]+', '', post_text)
        comments = json_data['comments']
        for comment in comments:
            comment_text = unescape(comment['text'])
            comment_text = re.sub("'", '"', comment_text) #заменяю ' на ", чтобы убрать экранирование \'
            comment_text = comment_text.replace('"', "'") #обратная замена
            for pattern, replacement in smileys.items():
                comment_text = re.sub(pattern, replacement, comment_text)
            # Используем регулярное выражение для поиска всех ссылок в тексте
            urls = re.findall(r'(http\S+?)(?=(http|\s|$))', comment_text) 
            # Используем цикл, чтобы заменить каждую ссылку в тексте на эту же ссылку с пробелами по обе стороны
            for url in urls:
                comment_text = comment_text.replace(url[0], f' {url[0]} ')
            comment_text = re.sub(r'[^\x00-\x7F]+', '', comment_text) #убирает непонятные символы unicode 
            comment_text = re.sub(r'\s+', ' ',  comment_text.strip()) #убирает все лишние пробелы
            comment_text = re.sub(r'(https?://\S+|www\.\S+)', 'URL', comment_text) #заменяет ссылки http,https, www на URL
            #comment_text = re.sub(r'[^\w\s]|_', '', comment_text) # очистка текста от всех символов, которые не являются алфавитно-цифровыми, лишних пробелов и знаков препинания
            score = comment.get('score')
            data.append({'post_text': post_text, 'comment': comment_text , 'score': score})

df_train = pd.DataFrame(data)

In [56]:
df_train

,post_text,comment,score
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4
...,...,...,...
440530,Pay your rent with a Credit or Debit card. No ...,Most major banks offer a service called 'bill ...,0
440531,Pay your rent with a Credit or Debit card. No ...,"It costs 3.25%, or $74.25 for the example of $...",1
440532,Pay your rent with a Credit or Debit card. No ...,As many other comments have pointed out almost...,2
440533,Pay your rent with a Credit or Debit card. No ...,My apartment building uses Yapstone's RentPaym...,3


# Предобрабокта 

In [57]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [59]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer

# загружаем стоп-слова
stop_words = set(stopwords.words('english'))

# инициализируем стеммер и лемматизатор
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# токенизируем и очищаем текст
def preprocess_text(text):
    # токенизация
    tokens = word_tokenize(text.lower())
    
    # удаление стоп-слов и пунктуации
    tokens = [token for token in tokens if token not in stop_words and token.isalpha()]
    
    # стемминг
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    # лемматизация
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return stemmed_tokens, lemmatized_tokens

# Для train 19 минут примерно

In [60]:
for d in data:
    # токенизация, стемминг, лемматизация для 'post_text'
    stemmed_tokens,lemmatized_tokens = preprocess_text(d['post_text'])
    d['post_stemmed_tokens'] = ' '.join(stemmed_tokens)
    d['post_lemmatized_tokens'] = ' '.join(lemmatized_tokens)

     # токенизация, стемминг, лемматизация для 'comment'
    stemmed_tokens, lemmatized_tokens = preprocess_text(d['comment'])
    d['comment_stemmed_tokens'] = ' '.join(stemmed_tokens)
    d['comment_lemmatized_tokens'] = ' '.join(lemmatized_tokens)

In [62]:
df_train = pd.DataFrame(data)

# GLOVE - *получился*

In [63]:
import numpy as np
import pandas as pd
import gensim.downloader as api

# Собираем все уникальные слова из текстов post_stemmed_tokens и comment_stemmed_tokens
words = set()
for d in data:
    words.update(d['post_lemmatized_tokens'].split())
    words.update(d['comment_lemmatized_tokens'].split())

# Загружаем модель GloVe
model = api.load('glove-wiki-gigaword-50')

# Создаем словарь, где каждому слову соответствует его векторное представление в модели GloVe
word_vectors = {}
for word in words:
    try:
        word_vectors[word] = model[word]
    except KeyError:
        word_vectors[word] = np.zeros(model.vector_size)

# Создаем новый столбец 'word_vectors' в DataFrame
df_train['word_vectors_glove'] = df_train.apply(lambda row: [word_vectors[word] for word in row['post_lemmatized_tokens'].split() + row['comment_lemmatized_tokens'].split()], axis=1)


In [64]:
df_train

,post_text,comment,score,post_stemmed_tokens,post_lemmatized_tokens,comment_stemmed_tokens,comment_lemmatized_tokens,word_vectors_glove
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0,mani summer combin funde decid continu startup...,many summer combinator fundees decided continu...,go back school ident give founder go back scho...,going back school identical giving founder go ...,"[[0.6979, 0.08234, 0.041526, -0.50704, -0.1580..."
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1,mani summer combin funde decid continu startup...,many summer combinator fundees decided continu...,invari see success set fall back origin path f...,invariably see success set fall back original ...,"[[0.6979, 0.08234, 0.041526, -0.50704, -0.1580..."
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2,mani summer combin funde decid continu startup...,many summer combinator fundees decided continu...,school way connect go world enter school think...,school way connected going world entered schoo...,"[[0.6979, 0.08234, 0.041526, -0.50704, -0.1580..."
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3,mani summer combin funde decid continu startup...,many summer combinator fundees decided continu...,guess realli depend hungri much believ product...,guess really depends hungry much believe produ...,"[[0.6979, 0.08234, 0.041526, -0.50704, -0.1580..."
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4,mani summer combin funde decid continu startup...,many summer combinator fundees decided continu...,know pollground decid go back school get combi...,know pollground decided go back school getting...,"[[0.6979, 0.08234, 0.041526, -0.50704, -0.1580..."
...,...,...,...,...,...,...,...,...
440530,Pay your rent with a Credit or Debit card. No ...,Most major banks offer a service called 'bill ...,0,pay rent credit debit card landlord signup requir,pay rent credit debit card landlord signup req...,major bank offer servic call pay free directli...,major bank offer service called pay free direc...,"[[0.6349, 0.55423, 0.93537, -0.47755, 0.24893,..."
440531,Pay your rent with a Credit or Debit card. No ...,"It costs 3.25%, or $74.25 for the example of $...",1,pay rent credit debit card landlord signup requir,pay rent credit debit card landlord signup req...,cost exampl look rack point best pump payment ...,cost example looking rack point best pumping p...,"[[0.6349, 0.55423, 0.93537, -0.47755, 0.24893,..."
440532,Pay your rent with a Credit or Debit card. No ...,As many other comments have pointed out almost...,2,pay rent credit debit card landlord signup requir,pay rent credit debit card landlord signup req...,mani comment point almost everyon debit card a...,many comment pointed almost everyone debit car...,"[[0.6349, 0.55423, 0.93537, -0.47755, 0.24893,..."
440533,Pay your rent with a Credit or Debit card. No ...,My apartment building uses Yapstone's RentPaym...,3,pay rent credit debit card landlord signup requir,pay rent credit debit card landlord signup req...,apart build use yapston charg flat fee use cre...,apartment building us yapstone charge flat fee...,"[[0.6349, 0.55423, 0.93537, -0.47755, 0.24893,..."


# BOW - МЕШОК СЛОВ (не получился, вылетает)

In [69]:
from sklearn.feature_extraction.text import CountVectorizer

# Создаем объект CountVectorizer
vectorizer = CountVectorizer()

# Объединяем лемматизированные токены из столбцов 'post_lemmatized_tokens' и 'comment_lemmatized_tokens'
merged_tokens = df_train['post_lemmatized_tokens'] + ' ' + df_train['comment_lemmatized_tokens']

# Преобразуем текстовые данные в матрицу Bag of Words
bow_matrix = vectorizer.fit_transform(merged_tokens)

# Получаем список всех слов в порядке, соответствующем их индексам в матрице Bag of Words
feature_names = list(vectorizer.vocabulary_.keys())

# Выводим размерность матрицы Bag of Words и первые 10 слов в словаре
print(f'Shape of Bag of Words matrix: {bow_matrix.shape}')
print(f'First 10 words in the dictionary: {feature_names[:10]}')


Shape of Bag of Words matrix: (440535, 169010)
First 10 words in the dictionary: ['many', 'summer', 'combinator', 'fundees', 'decided', 'continue', 'startup', 'go', 'back', 'school']


# Вот тут не запускается (вылетает)

In [ ]:
# Получаем массив признаков из разреженной матрицы
feature_matrix = bow_matrix.toarray()

# Добавляем новый столбец в DataFrame
df_train['bow_matrix'] = feature_matrix.tolist()


# TFIDF (не получился вылетает)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Создаем объект TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Объединяем лемматизированные токены из столбцов 'post_lemmatized_tokens' и 'comment_lemmatized_tokens'
merged_tokens = df_train['post_lemmatized_tokens'] + ' ' + df_train['comment_lemmatized_tokens']

# Преобразуем текстовые данные в матрицу TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(merged_tokens)

# Получаем список всех слов в порядке, соответствующем их индексам в матрице TF-IDF
feature_names = list(tfidf_vectorizer.vocabulary_.keys())

# Выводим размерность матрицы TF-IDF и первые 10 слов в словаре
print(f'Shape of TF-IDF matrix: {tfidf_matrix.shape}')
print(f'First 10 words in the dictionary: {feature_names[:10]}')


Shape of TF-IDF matrix: (439043, 167078)
First 10 words in the dictionary: ['many', 'summer', 'combinator', 'fundees', 'decided', 'continue', 'startup', 'go', 'back', 'school']
